In [108]:
import torch

In [162]:
x = torch.tensor([[1,2,3],[3,4,5]],dtype=torch.float, requires_grad=True)
w = torch.tensor([2,3], dtype=torch.float, requires_grad=True)
y = x*w[:,None]
z = y[torch.arange(2).long(), torch.arange(2).long()]
f = z.sum()

In [163]:
f.retain_grad()
y.retain_grad()
z.retain_grad()

In [164]:
f.backward()

In [165]:
f.grad

tensor(1.)

In [166]:
z.grad

tensor([ 1.,  1.])

In [167]:
y.grad

tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.]])

In [168]:
w.grad

tensor([ 1.,  4.])

In [169]:
x.grad

tensor([[ 2.,  0.,  0.],
        [ 0.,  3.,  0.]])

In [170]:
w

tensor([ 2.,  3.])

In [171]:
x

tensor([[ 1.,  2.,  3.],
        [ 3.,  4.,  5.]])

In [172]:
y

tensor([[  2.,   4.,   6.],
        [  9.,  12.,  15.]])

In [173]:
z

tensor([  2.,  12.])

In [174]:
f

tensor(14.)

In [175]:
f.grad

tensor(1.)

In [176]:
z.grad

tensor([ 1.,  1.])

In [177]:
y.grad

tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.]])

In [178]:
w.grad

tensor([ 1.,  4.])

In [179]:
x.grad

tensor([[ 2.,  0.,  0.],
        [ 0.,  3.,  0.]])

In [182]:
x

tensor([[ 1.,  2.,  3.],
        [ 3.,  4.,  5.]])

In [28]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [30]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Clear previously loaded data.
Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


In [211]:
X = X_dev
y = y_dev
W = np.random.randn(3073, 10) * 0.0001

W = torch.from_numpy(W)
X = torch.from_numpy(X)
W.requires_grad = True
wx = torch.mm(X,W)
#p_unnorm = torch.exp(wx-wx.max(1)[0][:,None])
p_unnorm = torch.exp(wx)
p_sum = torch.sum(p_unnorm,1)[:,None]
p_sum_inv = 1/p_sum
p = p_unnorm * p_sum_inv
L = p[torch.arange(500).long(),torch.Tensor(1).long()].sum()

In [212]:
wx.retain_grad()
p_unnorm.retain_grad()
p_sum.retain_grad()
p_sum_inv.retain_grad()
p.retain_grad()
L.retain_grad()

In [213]:
L.backward()

In [214]:
p.grad

tensor([[ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.]], dtype=torch.float64)

In [215]:
p_unnorm.shape

torch.Size([500, 10])

In [216]:
p.shape

torch.Size([500, 10])

In [217]:
p_sum_inv.shape

torch.Size([500, 1])

In [218]:
p.grad

tensor([[ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.]], dtype=torch.float64)

In [222]:
(p.grad*p_unnorm)[0]

tensor([ 0.9971,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], dtype=torch.float64)

In [223]:
p_sum_inv.grad[0]

tensor([ 0.9971], dtype=torch.float64)

In [221]:
p_unnorm.grad[0]

tensor(1.00000e-02 *
       [ 8.2950, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303,
        -0.8303, -0.8303, -0.8303], dtype=torch.float64)

In [75]:
p_sum_inv.grad[0]

tensor([ 0.3238], dtype=torch.float64)

In [76]:
p_unnorm[0]

tensor([ 0.3238,  1.0000,  0.1879,  0.1327,  0.4738,  0.3038,  0.6284,
         0.6004,  0.6733,  0.3476], dtype=torch.float64)

In [77]:
p_sum_inv[0]

tensor([ 0.2141], dtype=torch.float64)

In [225]:
p_sum.grad[0]

tensor(1.00000e-03 *
       [-8.3031], dtype=torch.float64)

In [226]:
p_sum.grad[0] + (p.grad*p_sum_inv)[0]

tensor(1.00000e-02 *
       [ 8.2950, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303,
        -0.8303, -0.8303, -0.8303], dtype=torch.float64)

In [227]:
p_unnorm.grad[0]

tensor(1.00000e-02 *
       [ 8.2950, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303, -0.8303,
        -0.8303, -0.8303, -0.8303], dtype=torch.float64)

In [37]:
dL = 1
dp_log = torch.zeros_like(p_log)
dp_log[:] = -1/len(dp_log)
dyp = 1/yp
dp = torch.zeros_like(p)
idx = torch.LongTensor(y)
j = torch.arange(dp.size(0)).long()
dp[j, idx] = 1
dp_unnorm = p_sum_inv

tensor(500.0000, dtype=torch.float64)